# Setup


Import nltk y descargar stopwords de la pestaña de corpus. EJECUCIÓN UNICA. 

In [ ]:
import nltk
nltk.download()
# Download stopwords from corpus

In [ ]:
from tika import parser #pip install tika REQUIERE JDK
import os
import unidecode #pip install unidecode
import time
#Import TfIdfVectorizer from scikit-learn
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.corpus import stopwords
import pandas as pd

Esta sección parsea los datos de los pdf a los .txt

In [ ]:

directory = "data/planes_materias/pdf/"

for filename in os.listdir(directory):
    if filename.endswith(".pdf"): 
        print(filename)
        raw = parser.from_file(os.path.join(directory, filename))
        #print(raw['content'])
        lines = raw['content']

        lines = unidecode.unidecode(lines[77:])
        #print(lines)
        f = open(os.path.join("data/planes_materias/txt/", filename[:-3]+ "txt"), 'w')
        f.write(lines)
        f.close()
        time.sleep(.9)
        continue
    else:
        continue

# Aqui empieza el main

The problem at hand is a Natural Language Processing problem. Hence you need to extract some kind of features from the above text data before you can compute the similarity and/or dissimilarity between them. To put it simply, it is not possible to compute the similarity between any two overviews in their raw forms. To do this, you need to compute the word vectors of each overview or document, as it will be called from now on.

As the name suggests, word vectors are vectorized representation of words in a document. The vectors carry a semantic meaning with it. For example, man & king will have vector representations close to each other while man & woman would have representation far from each other.

You will compute Term Frequency-Inverse Document Frequency (TF-IDF) vectors for each document. This will give you a matrix where each column represents a word in the overview vocabulary (all the words that appear in at least one document), and each column represents a movie, as before.

https://www.datacamp.com/community/tutorials/recommender-systems-python

In [ ]:

directory = "data/planes_materias/txt/"

materias = {}
#print("hola")
for filename in os.listdir(directory):
    if filename.endswith(".txt"):
        #print(filename) 
        f = open(os.path.join(directory, filename), 'r')
        materias[filename[:-4]] = f.read()

#print(materias['1.txt'])


In [94]:
#Palabras que no consideraremos.
stop_words = stopwords.words('spanish')
my_stop_words = ["CARRERA","MATEMATICO","SERIACION", "INDICATIVA" ,"ANTECEDENTE","MODALIDAD","CURSO","CARACTER", "TEORICAS", "PRACTICAS" ,"CREDITOS", "HORAS", "SEMANA", "SEMESTRE", "CLAVE", "FACULTAD", "CIENCIAS", "OBJETIVO","BIBLIOGRAFIA", "BASICA", "COMPLEMENTARIA","SUGERENCIA","PARA","LA","EVALUACION","DE","LA","ASIGNATURA","Ademas","de","las","calificaciones",
"en","examenes","y","tareas","se","tomara","en","cuenta","la","participacion","del","alumno",
"PERFIL","PROFESIOGRAFICO","Matematico","fisico","actuario","o","licenciado","en","ciencias"
,"de","la","computacion","especialista","en","el","area","de","la","asignatura","a","juicio"
,"del","comite","de","asignacion","de", "cursos","UNIDADES", "TEMATICAS","SERIACION" ,"INDICATIVA" ,"ANTECEDENTE", "SUBSECUENTE","optativo","Optativas"]
stop_words.append(my_stop_words)

#data = pd.DataFrame(materias)
data = pd.Series(materias).to_frame()
type(data)
data = data.rename(columns={0: 'Planes'})
#data["index"] = pd.to_numeric(data["index"])
data.index = data.index.map(int)
data.sort_index(inplace=True)

#print(data.columns)
print(data.index)


#Define a TF-IDF Vectorizer Object. Remove all spanis stop words such as 'de', 'la', 'que', 'el'
tfidf = TfidfVectorizer(stop_words)

#Construct the required TF-IDF matrix by fitting and transforming the data
tfidf_matrix = tfidf.fit_transform(data['Planes'])

#Output the shape of tfidf_matrix
tfidf_matrix.shape


Int64Index([   1,    2,    3,    4,    5,    6,    7,    8,    9,   10,
            ...
             990,  991, 1063, 1506, 1609, 1706, 1707, 1710, 1712, 1815],
           dtype='int64', length=154)


(154, 9694)

In [95]:
#Palabras encontradas en todos los archivos
tfidf.get_feature_names()[2990:3007]

['digraficas',
 'digraphs',
 'dijkstra',
 'dilatacion',
 'dilke',
 'dillon',
 'dimension',
 'dimensional',
 'dimensionales',
 'dimensionality',
 'dimensionantropologica',
 'dimensiones',
 'dinamica',
 'dinamical',
 'dinamicas',
 'dinamico',
 'dinamicos']

In [96]:
from sklearn.metrics.pairwise import linear_kernel

# Compute the cosine similarity matrix
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(154, 154)

In [97]:
cosine_sim[1]

array([0.40419113, 1.        , 0.31810136, 0.3515157 , 0.26657346,
       0.31713414, 0.30480452, 0.31898517, 0.25845382, 0.30743276,
       0.24300932, 0.21455324, 0.22938134, 0.22066034, 0.21416592,
       0.2454753 , 0.23546273, 0.28656018, 0.29716748, 0.25598118,
       0.2855816 , 0.2716472 , 0.26014019, 0.23422356, 0.2204104 ,
       0.23120536, 0.24211392, 0.19774053, 0.3271041 , 0.24020937,
       0.21308072, 0.26885764, 0.27978719, 0.25689672, 0.22451863,
       0.27678151, 0.27720559, 0.23918952, 0.2131673 , 0.24796128,
       0.20746668, 0.28957904, 0.23136835, 0.28661217, 0.24575322,
       0.17456442, 0.26137049, 0.18708388, 0.25309904, 0.15546245,
       0.24857502, 0.2185814 , 0.24563228, 0.26062098, 0.21728521,
       0.24633038, 0.25785203, 0.21570145, 0.18144005, 0.26099365,
       0.21756244, 0.26167192, 0.21205772, 0.20923217, 0.24056683,
       0.22064788, 0.2653997 , 0.21544989, 0.11230661, 0.24403525,
       0.25197591, 0.23473167, 0.26014644, 0.17353269, 0.27628

In [105]:
tfidf_matrix = tfidf.transform(["Topologia", "Geometria"])
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
cosine_sim.shape

(2, 2)

In [106]:
#indices = pd.Series(data.index, index=data.index).drop_duplicates()

# Function that takes in movie title as input and outputs most similar movies
def get_recommendations(string, cosine_sim=cosine_sim):
    # Get the index of the movie that matches the title
    

    # Get the pairwsie similarity scores of all movies with that movie
    sim_scores = list(enumerate(cosine_sim[string]))
    

    # Sort the topics based on the similarity scores
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    #print(sim_scores)
    # Get the scores of the  10most similar topics
    sim_scores = sim_scores[0:11]

    # Get the movie indices
    recomendaciones = [i[0] for i in sim_scores]
    print(recomendaciones)



    # Return the top 10 most similar topics
    return data.iloc[recomendaciones]

In [103]:
# Devuelve una lista de 10 recomendaciones de la posicion 5 de los temarios. 

# Poner posicion, no clave.
#1.pdf -> 0
#2.pdf -> 1

Foo(clave ) -> index


get_recommendations(1)

[1, 0, 3, 93, 28, 130, 7, 2, 5, 104, 102]


,Planes
2,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
1,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
4,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
712,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
242,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
939,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
8,A DE MEXICO \n\n \nFacultad de Ciencias \n\n \...
3,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
6,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
745,FACULTAD DE CIENCIAS\n\nCARRERA DE MATEMATICO\...
